In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Configurações de conexão
user = 'luiz'
password = 't5NajFQr9COpyP'
host = 'db.nqriypvbuemqemsuhisk.supabase.co'
port = 5432
database = 'postgres'

# String de conexão
conn_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(conn_str)

# Substitua 'nome_da_tabela' pela tabela que você deseja consultar
query1 = 'SELECT * FROM review'
query2 = 'SELECT * FROM aspect'

# Carregar os dados em um DataFrame pandas
df = pd.read_sql(query1, engine)
df.dropna(subset=["review_rating", "review_text"], inplace=True)
df = df[df["review_text"].str.strip() != ""]
df["review_posted_at"] = pd.to_datetime(df["review_posted_at"], errors='coerce')

df2 = pd.read_sql(query2, engine)
df2.dropna(subset=["name"], inplace=True)
df2 = df2[df2["name"].str.strip() != ""]

df_final = pd.merge(df, df2, right_on="review_id", left_on="id", how="inner")

# Exibir o DataFrame
df2


,id,name,category,description,sentiment,intention,review_id,taxonomy_category,taxonomy_name
1,15b3d5dd-ca8c-46f9-ab84-43ac5a3fd832,Limpeza,Quarto,Super limpio,1,ELOGIO,cc8c709d-6b61-422c-af69-7fc195b38848,Quarto,Limpeza
10,56375463-b2f6-44d9-ace5-943637cf368b,Limpeza,Quarto,é limpo,1,ELOGIO,943e39b5-5aa1-46c6-b1fd-aa25bf40d199,Quarto,Limpeza
13,e4c5f0fa-724b-4522-8821-6dabb6469195,Ar-condicionado/climatização,Quarto,exceto pelo ar condicionado que pinga a noite ...,-1,RECLAMAÇÃO,c50248c2-6861-44c1-9fc0-d86f432eaa8b,Quarto,Ar-condicionado/climatização
14,91185d18-e9b9-4efd-8778-540fd574350b,Limpeza,Quarto,a limpeza poderia ser melhor do quarto e banhe...,-1,RECLAMAÇÃO,c50248c2-6861-44c1-9fc0-d86f432eaa8b,Quarto,Limpeza
15,bf6b8fa8-a2f3-4c91-b1a0-87fad3b41279,Qualidade,Restaurante/Bar,máquina do café na recepção falha..não tem pro...,-1,RECLAMAÇÃO,c50248c2-6861-44c1-9fc0-d86f432eaa8b,Restaurante/Bar,Qualidade
...,...,...,...,...,...,...,...,...,...
49881,68516ab1-c048-4605-b3db-4e22369d14ff,Academia,Áreas comuns,"tem área de jogos, academia funcional, piscina...",1,ELOGIO,ba64cc44-402b-4b29-aaae-950c719a7365,Áreas comuns,Academia
49882,6de2399c-dc0b-4cd9-ae10-d3181e17a6c7,Piscina,Áreas comuns,"tem área de jogos, academia funcional, piscina...",1,ELOGIO,ba64cc44-402b-4b29-aaae-950c719a7365,Áreas comuns,Piscina
49883,2ef22e22-3c34-41a7-ba7b-bcc9517315bb,Limpeza,Quarto,O hotel é limpo,1,ELOGIO,ba64cc44-402b-4b29-aaae-950c719a7365,Quarto,Limpeza
49886,a7fdbcff-6c03-433f-9401-d880ddb817b2,Educação e cordialidade,Atendimento da equipe,Gostaria de agradecer a atendente Fabíola pela...,1,ELOGIO,f69f6a37-784b-4cb9-8d58-0784336b17b1,Atendimento da equipe,Educação e cordialidade


In [5]:
df.to_csv('review_data.csv', index=False)

In [2]:
df2.to_csv('aspect_data.csv', index=False)

In [10]:
df_for_text_sentiment = df_final[["description", "sentiment", "category", "hotel_id", "user_language", "review_posted_at"]].copy()

for hotel in df_for_text_sentiment["hotel_id"].unique():
    df_hotel = df_for_text_sentiment[df_for_text_sentiment["hotel_id"] == hotel]
    print(df_hotel["sentiment"].unique())

[ 1 -1  0]
[ 1 -1  0]
[ 1 -1  0]
[ 1 -1  0]
[ 1 -1  0]
[-1  1  0]
[-1  1  0]
[ 1 -1  0]


In [11]:
df_for_text_sentiment = df_for_text_sentiment[df_for_text_sentiment["sentiment"] != 0].copy()
df_for_text_sentiment.loc[:, "sentiment"] = df_for_text_sentiment["sentiment"].replace(-1, 0)
df_for_text_sentiment["sentiment"].value_counts()

sentiment
1    25202
0     8895
Name: count, dtype: int64

In [12]:
df_for_text_sentiment = df_for_text_sentiment[df_for_text_sentiment["user_language"] == "pt"]
df_for_text_sentiment

,description,sentiment,category,hotel_id,user_language,review_posted_at
0,ar condicionado bem silencioso,1,Quarto,viale_cataratas,pt,2025-01-19 00:00:00+00:00
1,camas boas,1,Quarto,viale_cataratas,pt,2025-01-19 00:00:00+00:00
2,"Copa baby bem abandonada, estava suja, não tin...",0,Áreas comuns,viale_cataratas,pt,2025-01-19 00:00:00+00:00
5,Hotel com ótima estrutura,1,Experiência,viale_cataratas,pt,2025-01-17 00:00:00+00:00
6,camas confortáveis,1,Quarto,viale_cataratas,pt,2025-01-17 00:00:00+00:00
...,...,...,...,...,...,...
34626,super bem atendida por todos ❤️,1,Atendimento da equipe,taroba,pt,2024-07-08 00:50:16+00:00
34627,"Maravilhosa, lugar lindo",1,Experiência,taroba,pt,2024-07-08 00:50:16+00:00
34628,limpeza,1,Quarto,taroba,pt,2024-01-09 01:40:17+00:00
34629,"A camareira Lourdes com um trabalho impecável,...",1,Atendimento da equipe,taroba,pt,2024-01-07 18:43:27+00:00


In [13]:
df_for_text_sentiment["category"].value_counts()

category
Atendimento da equipe    3893
Quarto                   3256
Áreas comuns             1945
Café da manhã            1603
Restaurante/Bar          1322
Experiência              1234
Localização               950
Banheiro                  548
Segurança                  65
Experincia                  1
Estacionamento              1
Name: count, dtype: int64

In [14]:
df_for_text_sentiment.drop(["user_language"], axis=1, inplace=True)

In [15]:
df_final = pd.DataFrame()

for hotel in df_for_text_sentiment["hotel_id"].unique():
    df_hotel = df_for_text_sentiment[df_for_text_sentiment["hotel_id"] == hotel]
    df_hotel.reset_index(drop=True, inplace=True)

    categories_to_keep = df_hotel["category"].value_counts()[lambda x: x >= 100].index
    df_hotel = df_hotel[df_hotel["category"].isin(categories_to_keep)].copy()

    df_final = pd.concat([df_final, df_hotel], ignore_index=True)

df_final["category"].value_counts()

category
Atendimento da equipe    3893
Quarto                   3256
Áreas comuns             1859
Café da manhã            1426
Experiência               945
Restaurante/Bar           926
Localização               740
Banheiro                  150
Name: count, dtype: int64

In [16]:
import os

# Save individual CSVs for each hotel with reset index
for h in df_final["hotel_id"].unique():
    folder_path = "datasets/"
    os.makedirs(folder_path, exist_ok=True)
    
    # Filter reviews for the current hotel and reset the index
    df_h = df_final[df_final["hotel_id"] == h].reset_index(drop=True)
    
    # Save CSV file in the corresponding folder
    csv_filename = f"{h}.csv"
    csv_path = os.path.join(folder_path, csv_filename)
    df_h.to_csv(csv_path, index=False)